# Import data

In [ ]:
from fredapi import Fred
import plotly.graph_objects as go #used for data visualization
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

fred = Fred(api_key='0e929d96cfbf2133c9a346a74670f033')
df = pd.DataFrame(fred.get_series('DPSACBW027NBOG')).reset_index()

# renaming columns
df.columns = ['report_date', 'val']

# only include dates from 2017+ 
df = df[df['report_date'] >= '2017-01-01']

# visualizing 
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['report_date'], y=df['val'], name='Total Deposits'))

In order to use holts, we need to remove the seasonal and trend components of the data.
* To remove the trend, we will difference the data.
* To remove the seasonal component, we will use fourier series.
    * 2 ways of Seasonally adjusting the data:
        1. removing the seasonal component from the original data
        2. removing the seasonal component from the differenced data

In [ ]:
# REMOVE TREND by differencing
df['val change'] = df['val'].diff()

# REMOVE SEASONALITY by seasonal decomposing the data
result = seasonal_decompose(df['val'], model='additive', period=52)

result2 = seasonal_decompose(df['val change'].dropna(), model='additive', period=52)

df['seasonal_1'] = result.seasonal
df['seasonal_2'] = result2.seasonal

df['val change (SA1)'] = df['val change'] - df['seasonal_1']
df['val change (SA2)'] = df['val change'] - df['seasonal_2']

# dropna values
df = df.dropna()
df

# Simple Exponential Smoothing

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import timedelta
from tslearn.models import SES


# Create an instance of ExponentialSmoothing
model = SES(df, df['val change (SA1)'], trend='add', seasonal='add', seasonal_periods=12)

# fit = model.fit()

# forecast_values = fit.forecast(steps=12)

# # Convert forecasted values to a DataFrame and reset the index
# forecast_values_df = pd.DataFrame(forecast_values).reset_index()

# # Rename columns and add the 'report_date' column
# forecast_values_df.columns = ['index', 'val']
# forecast_values_df['report_date'] = pd.date_range(start=df['report_date'].max(), periods=len(forecast_values_df), freq='W')

# # Reorder columns to have 'report_date' and 'val'
# forecast_values_df = forecast_values_df[['report_date', 'val']]

# # Display the DataFrame
# forecast_values_df

# forecast_values
# model.get_df()

model.forecast(steps=12)